# Tokenize text

El primer paso será configurar una función que convierta el texto de entrada a tokens individuales, como paso de preprocesamiento para crear embeddings.

Se utiliza el text The Veredict que ha sido cedidio para poder entrenar LLM. https://en.wikisource.org/wiki/The_Verdict

In [5]:
with open("the-veredict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()

print("The text has", len(raw_text), "characters")
print(raw_text[:99])

The text has 20480 characters
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Para separar las palabras del texto en pos de tokenizarlas podemos utilizar re.split para separar las palabras en función de los espacios en blanco que dejan. En este caso se están separando las palabras en algunos casos con signos de puntuación, lo que querremos corregir más adelante. En este caso, a diferencia de otras pruebas que hemos realizado, se mantiene la capitalización de las palabras porque sirve para que el modelo pueda distinguir entre nombres propios y comunes, entender la estructura de la oración y para que genere texto correctamente capitalizado.

In [6]:
import re
text = 'Hello, world. This, is a test.'
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


El siguiente código permite separar también los signos de puntuación, y en la celda a continuación eliminaremos los espacios en blanco. Esta decisión va a depender del tipo de modelo que se esté entrenando y de nuestros recursos disponibles, ya que aunque reducen los requerimientos, alugnos modelos son muy sensibles a la estructura del texto y a los espacios e indentaciones.

In [7]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [8]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


Añadimos más tipos de puntuación que puedan aparecer en el texto.

In [9]:
text = 'Hello, world. This, is a test?. -- This is the end.'
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '?', '.', '--', 'This', 'is', 'the', 'end', '.']


Una vez definido el tokenizador básico se puede aplicar al texto "The Veredict".

In [10]:
def tokenizador(text): 
    result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    return [item for item in result if item.strip()]

preprocessed = tokenizador(raw_text)
print(len(preprocessed))

4690


In [11]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


# Tokens to ID

Para poder mapear los tokens a IDs hay que construir un vocabulario primero, que define como se mapea cada palabra y carácter único a un número entero.

En el ejemplo que estamos trabajando construimos una lista de todos los tokens únicos y los ordenamos alfabéticamente para determinar el tamaño del vocabulario.

In [12]:
words = sorted(list(set(preprocessed)))
vocab_size = len(words)
print(vocab_size)

vocab = {token: i for i, token in enumerate(words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


Con este diccionario podemos convertir el texto en IDs y, posteriormente invertir la versión del vocabuilario para reconvertir de ID a texto.

Creamos una clase que incluya el tokenizador, el encoder y el decoder para poder palicarlo al texto.

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def tokenizador(self, text):
        result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        return [item for item in result if item.strip()]

    def encode(self, text):
        preprocessed = tokenizador(text)
        ids = [self.str_to_int[word] for word in preprocessed]
        return ids

    def decode (self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?_!"()\']|--)', r'\1', text)
        return text

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

Probamos que el encoder funciona correctamente

In [15]:
text = """"It's the last he painted, you know," Mrs. Gisburn said"""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851]


Probamos ahora que podemos recuperar el texto tras codificarlo

In [17]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said


El problema sucede cuando tratamos de aplicar el tokenizador con palabras que no figuran en el texto de entrenamiento, por ejemplo:

In [18]:
text = "Hello world?"
tokenizer.encode(text)

KeyError: 'Hello'

Por ello se requieren de sets de entrenamientos muy amplios.

# Tokens especiales de contexto

En este punto se añadirán funcioanlidades al tokenizador para que pueda trabajar con vocabulario que no tenga disponible, y además generar tokens especiales que puedan dar información contextual relevante.

Se van a añadir por tanto dos tokens, <|unk|> para palabras desconocidas por el vocabulario y <|endoftext|> para señalar que el documento/libro/texto ha finalizado y no tiene relación con el siguiente texto.

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)


('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [29]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in result if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[word] for word in preprocessed]
        return ids

    def decode (self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?_!"()\']|--)', r'\1', text)
        return text

In [32]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace"
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace


In [ ]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131]


In [34]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>


Dependiendo del tipo de LLM otros tokens recomendados son:
- [BOS]: Beginning of sequence
- [EOS]: End of sequence
- [PAD]: Padding, asegura que todos los textos tienen la misma longitud, por lo que los textos más cortos se extienden usando [PAD].

El tokenizador de GPT por ejemplo no tuiliza ninguno de estos tokens y solo utiliza endoftext.

# Byte pair encoding

Se trata de una forma mas sofisticada de tokenización que la que hemos estado usando de ejemplo, este tokenizador se ha usado para entrenar los GPT 2 y 3.

Para implementarlo se utiliza la libreria tiktoken que ya implementa el algoritmo BPE usando codigo fuente de Rust. 

Podemos instanicar el BPE tokenizer cogiendo directamente el método de codificación de gpt2.

In [35]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [38]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace"
text = " <|endoftext|> ".join((text1, text2))

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 262, 20562]


In [39]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace


El tokenizador BPE de los modelos GPT tiene un tamaño de vocabuilario de 50257, donde se asigna <|endoftext|> al valor más grande.

El algoritmo de BPE además puede trabajar con palabras desconocidas transformando estas palabras en subpalabras más pequeñas, incluso a caractéres individuales, de tal forma puede trabajar con palabras incluso desconocidas para poder trabajar con ellas.

Su vocabulario se construye uniendo caractéres frecuentes en subpalabras y subpalabras frecuentes en palabras, todo de forma iterativa.

# Data sampling

El siguiente paso es generar los pares de input-target que se requieren para entrenar un LLM.

Se implementará un cargador de datos que extrae los pares de input-target del conjunto de entrenamiento de datos utilizando un método "sliding-window".

In [40]:
with open("the-veredict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5146


In [42]:
enc_sample = enc_text[:50]

Una de las formas intuitivas para crear pares input-target para tareas de predicción de la siguiente palabra consiste en crear dos variables, x e y, donde x contiene los input tokens e y los targets, que serán los inputs movidos una posición.

In [47]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:     {y}")

x: [40, 367, 2885, 1464]
y:     [367, 2885, 1464, 1807]


De esta forma se pueden crear las tareas de predicción de siguiente palabra.

In [48]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[40] ----> 367
[40, 367] ----> 2885
[40, 367, 2885] ----> 1464
[40, 367, 2885, 1464] ----> 1807


In [50]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

I ---->  H
I H ----> AD
I HAD ---->  always
I HAD always ---->  thought


El último paso para convertir los tokens en embeddings es implementar un cargado de datos eficiente que itere sobre el dataset y devuelve los inputs y targets como tensores de PyTorch.

Necesitamos dos tensores, el input que es el texto que el LLM ve, y el target que incluye lo que el modelo debe predecir.

PyTorch incluye una clases Dataset y DataLoader que permiten la implementación eficiente.

In [65]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []

        tokens_ids = tokenizer.encode(txt)

        for i in range(0, len(tokens_ids) - max_length, stride):
            input_chunk = tokens_ids[i:i+max_length]
            target_chunk = tokens_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

Esta clase, basada en la clase Dataset de PyTorch, define como las filas se extraen del dataset, cada fila consiste de un númer de token IDs (basado en una longitud máxima) asignada a un tensor de input. El target tensor contiene los targets correspondientes.

El código siguiente carga los inputs en batches utilizando el DataLoader de PyTorch

In [66]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    return dataloader

In [67]:
with open("the-veredict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle = False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


El primer batch contiene dos tensores, el primero guarda los input token IDs, y el segundo los target. Como la longitud máxima se establece en 4 cada tensor contiene 4 IDs, en general los LLMs se entrenan con inputs de al menos 256.

El 'stride' define cuantas posiciones se van a desplazar los inputs entre batches, emulando el método de sliding window.

El tamaño del batch va a ser un hiperparámetro a ajustar, ya que tamaños menores requieren menos memoria en el entrenamiento pero dan como resultado modelos con mayor ruido.

# Embeddings

El último paso en la preparación de inputs es convertir los tokens en vectores de embedding.

Los pesos de los embeddings se inicializan con valores aleatorios como paso preliminar. Este paso sirve como punto inicial del entrenamiento del LLM, posteriormente habrá que optimizarlos.

Los embeddings (representación vectorial continua) es necesaria ya que estos modelos utilizan redes neuronales profundas con retropropagación.

Para ilustrar estos embeddings, suponemos que disponemos de un vocabulario de 6 palabras, y queremos un embedding de tamaño 3 (GPT-3 utiliza 12288 dimensiones como referencia)

In [68]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


La matriz de pesos tiene 6 filas (tamaño del vocabulario) y 3 columnas (cada dimensión del embedding).

La capa de embeddings es a efectos una operación 'look-up' que recupera filas de la matriz de pesos del embedding usando el ID del token.

# Codificación de posición

Aunque lo visto previamente es aceptable para un LLM, estos modelos no tienen una noción de la posición o el orden de los tokens en una secuencia. La capa de embeddings tiene el mismo problema.

El mecanismo de auto-atención del modelo no usa la posición, pero es útil incorporar esa información al modelo.

Existen dos categorías para embeddings de posición, los relativos y los absolutos.

Los embeddings absolutos generan para cada token un embedding posicional específico hasta el final de los tokens. Mientas que los relativos se fijan en la distancia relativa entre tokens. El modelo aprende como de lejos están los tokens.

GPT utiliza absolutos optimizados durante el entrenamiento. 

In [71]:
output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInput shape:", inputs.shape)


Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Input shape: torch.Size([8, 4])


In [72]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


Para usar el método GPT de embeddings absolutos crearemos otra capa de embeddings con las mismas dimensiones que la capa de embeddings. 

In [73]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [75]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
